In [1]:
!git clone https://github.com/EmperorNao/wda.git
!pip install -r wda/requirements.txt

Cloning into 'wda'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 86 (delta 34), reused 76 (delta 24), pack-reused 0
Receiving objects: 100% (86/86), 71.84 KiB | 3.99 MiB/s, done.
Resolving deltas: 100% (34/34), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 32.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=7263e8ab7a374491120510d94aae84de1064ce853d306d87649458f2df8d8176
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
  Created wheel 

In [2]:
%run -i 'wda/dataset.py'

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
import codecs
from bs4 import BeautifulSoup
import time



LOCAL = "/content"
dataset = Dataset(
    path_to_groups={
        'test': join(LOCAL, "test_groups.csv"),
        'train': join(LOCAL, "train_groups.csv"),
    },
)

In [6]:
# dataset.from_csv("simple_data.csv")

In [11]:
dataset.from_csv(join("/content", 'extended_dataset.csv'))

In [12]:
dataset[:]

,Unnamed: 0,pair_id,group_id,doc_id,target,IsTrain,title,keywords,abstract,description,title_len
0,0,1,1,15731,0.0,1,ваз 21213 | замена подшипник ступица | нива,"замена подшипник ступица , технический описани...",NaN,"замена подшипник ступица , руководство ремонт ...",8
1,1,2,1,14829,0.0,1,"ваз 2107 оптом сочи . сравнить цена , купить п...","ваз 2107 оптом , россия , сочи , цена , предло...","ваз 2107 опт . продажа , поиск , поставщик маг...","ваз 2107 опт . продажа , поиск , поставщик маг...",14
2,2,3,1,15764,0.0,1,купить ступица лада калина 2 . трансмиссия - п...,"цена , замена , тюнинг , купить , продать , ст...",NaN,продажа запчасть ступица легковой грузовой авт...,16
3,3,4,1,17669,0.0,1,классика 21010 - 21074,NaN,NaN,NaN,4
4,4,5,1,14852,0.0,1,ступица нива — замена подшипник свой рука,NaN,NaN,передний ступица нива иметь свой особенность ....,7
...,...,...,...,...,...,...,...,...,...,...,...
28312,28312,28313,309,16637,NaN,0,ответ @ mail . ru : полезный кушать творог утр...,NaN,NaN,пользователь лили гейбиф задать вопрос категор...,18
28313,28313,28314,309,16759,NaN,0,творог . полезный свойство лечение творог . же...,творог лечебный полезный целебный свойство тво...,NaN,"творог обладать многий полезный , лечебный ( ц...",14
28314,28314,28315,309,15358,NaN,0,творог - полезный опасный свойство творог,творог свойство,NaN,"творог , химический состав , пищевой ценность ...",6
28315,28315,28316,309,17287,NaN,0,ответ @ mail . ru : полезный творог ?,NaN,NaN,пользователь кит кит задать вопрос категория п...,9


In [13]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

#Load AutoModel from huggingface model repository
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
model = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

tokenizer_config.json:   0%|          | 0.00/323 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [14]:
model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(120138, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inp

In [187]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import numpy as np
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score, classification_report

In [16]:
def measure_std(sim, first_sents):
    first_sim = sim[first_sents,:][:,first_sents]
    np.fill_diagonal(first_sim, 0)
    return first_sim.std(axis=0).mean()

In [17]:

import pandas as pd
title_data = dataset[:].iloc[:, 1:]#pd.read_csv('docs_titles.tsv', sep='\t', encoding='utf-8', lineterminator='\n')
# title_data[0]['title']
title_data.loc[0]['title']

train_groups = pd.read_csv('train_groups.csv')
print (train_groups.shape)
train_groups.head()

test_groups = pd.read_csv('test_groups.csv')
print (test_groups.shape)
test_groups.head()


(11690, 4)
(16627, 3)


,pair_id,group_id,doc_id
0,11691,130,6710
1,11692,130,4030
2,11693,130,5561
3,11694,130,4055
4,11695,130,4247


In [ ]:
# title_data = title_data.fillna('')

In [20]:
doc_to_title = title_data

In [29]:
train_data = pd.read_csv('train_groups.csv')
traingroups_titledata = {}
for i in tqdm(range(len(train_data))):
    new_doc = train_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    target = new_doc['target']
    title = doc_to_title.loc[doc_to_title['doc_id'] == doc_id, 'title'].iloc[0]
    if doc_group not in traingroups_titledata:
        traingroups_titledata[doc_group] = []
    traingroups_titledata[doc_group].append((doc_id, title, target))

100%|██████████| 11690/11690 [00:07<00:00, 1512.43it/s]


In [30]:
test_data = pd.read_csv('test_groups.csv')
testgroups_titledata = {}
for i in tqdm(range(len(test_data))):
    new_doc = test_data.iloc[i]
    doc_group = new_doc['group_id']
    doc_id = new_doc['doc_id']
    pair_id = new_doc['pair_id']
    title = doc_to_title.loc[doc_to_title['doc_id'] == doc_id, 'title'].iloc[0]
    if doc_group not in testgroups_titledata:
        testgroups_titledata[doc_group] = []
    testgroups_titledata[doc_group].append((doc_id, title, pair_id))

100%|██████████| 16627/16627 [00:07<00:00, 2362.27it/s]


In [31]:
len(traingroups_titledata)

129

In [101]:
import numpy as np
X_train_most_words = []
groups_train = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        groups_train.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_train_most_words.append(sorted(all_dist, reverse=True)[0:15]    )
X_train_most_words = np.array(X_train_most_words)
groups_train = np.array(groups_train)


X_test_most_words = []
groups_test = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]
    for k, (doc_id, title, target_id) in enumerate(docs):
        groups_test.append(new_group)
        all_dist = []
        words = set(title.strip().split())
        for j in range(0, len(docs)):
            if k == j:
                continue
            doc_id_j, title_j, target_j = docs[j]
            words_j = set(title_j.strip().split())
            all_dist.append(len(words.intersection(words_j)))
        X_test_most_words.append(sorted(all_dist, reverse=True)[0:15]    )
X_test_most_words = np.array(X_test_most_words)
groups_test = np.array(groups_test)

In [164]:
PUNKT = "|[]{},./\\\"\'`~"

top_k = 15

X_train_top_words = []
for new_group in traingroups_titledata:
    docs = traingroups_titledata[new_group]

    all_words = []
    words = set()
    for k, (doc_id, title, target_id) in enumerate(docs):
        cur_words = ' '.join(filter(lambda x: x not in PUNKT, title.strip().split()))
        all_words.append(cur_words)
        words = words & set(cur_words)

    cv = np.array(CountVectorizer().fit_transform(all_words).todense())
    counts = cv.sum(axis=0)

    idxs = np.argsort(counts)[::-1]
    ret = np.zeros((cv.shape[0], top_k))
    ret[:,:min(idxs.shape[0], top_k)] = cv[:, idxs[:min(idxs.shape[0], top_k)]]

    one_idxs = np.where(np.logical_and(counts >= 3, counts <= 10))[0]

    n_less_word = cv[:, one_idxs].sum(axis=1)[:, None]
    X_train_top_words.append(np.concatenate([ret, n_less_word], axis=1))

X_train_top_words = np.concatenate(X_train_top_words, axis=0)



X_test_top_words = []
for new_group in testgroups_titledata:
    docs = testgroups_titledata[new_group]

    all_words = []
    for k, (doc_id, title, target_id) in enumerate(docs):
        all_words.append(' '.join(filter(lambda x: x not in PUNKT, title.strip().split())))

    cv = np.array(CountVectorizer().fit_transform(all_words).todense())
    counts = cv.sum(axis=0)
    idxs = np.argsort(counts)[::-1]
    ret = np.zeros((cv.shape[0], top_k))
    ret[:,:min(idxs.shape[0], top_k)] = cv[:, idxs[:min(idxs.shape[0], top_k)]]

    one_idxs = np.where(np.logical_and(counts >= 3, counts <= 10))[0]

    n_less_word = cv[:, one_idxs].sum(axis=1)[:, None]

    X_test_top_words.append(np.concatenate([ret, n_less_word], axis=1))

X_test_top_words = np.concatenate(X_test_top_words, axis=0)

In [78]:
def encode_sents(sents):

    model.to('cuda')
    encoded_input = tokenizer(sents, padding=True, truncation=True, max_length=128, return_tensors='pt')

    with torch.no_grad():
        model_output = model(**encoded_input.to('cuda'))

    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask']).detach().cpu()

    return sentence_embeddings


def get_avg_sent(group_sents):
    return group_sents.mean(axis=0, keepdim=True)

In [189]:
y = []
X_embeddings = []
X_embeddings_cosine = []
X_embeddings_euclidean = []
X_texts = []
for group, data in traingroups_titledata.items():
    sents = []
    for dataitem in data:
        y.append(dataitem[2])
        sents.append(dataitem[1])

    emb = encode_sents(sents)
    avg_emb = get_avg_sent(emb)
    diff_embs = emb - avg_emb
    diff_embs_cosine = cosine_similarity(emb, avg_emb)
    diff_embs_euclidean = euclidean_distances(emb, avg_emb)

    X_texts.extend(sents)
    X_embeddings.extend(diff_embs.numpy())
    X_embeddings_cosine.extend(diff_embs_cosine)
    X_embeddings_euclidean.extend(diff_embs_euclidean)


X_embeddings = np.array(X_embeddings)
X_embeddings_euclidean = np.array(X_embeddings_euclidean)
X_embeddings_cosine = np.array(X_embeddings_cosine)


pairs = []
X_embeddings_test = []
X_embeddings_test_cosine = []
X_embeddings_test_euclidean = []
X_texts_test = []
for group, data in testgroups_titledata.items():

    sents = []
    for dataitem in data:
        pairs.append(dataitem[2])
        sents.append(dataitem[1])

    emb = encode_sents(sents)
    avg_emb = get_avg_sent(emb)
    diff_embs = emb - avg_emb

    diff_embs_cosine = cosine_similarity(emb, avg_emb)
    diff_embs_euclidean = euclidean_distances(emb, avg_emb)

    X_texts_test.extend(sents)
    X_embeddings_test.extend(diff_embs.numpy())
    X_embeddings_test_cosine.extend(diff_embs_cosine)
    X_embeddings_test_euclidean.extend(diff_embs_euclidean)

X_embeddings_test = np.array(X_embeddings_test)
X_embeddings_test_cosine = np.array(X_embeddings_test_cosine)
X_embeddings_test_euclidean = np.array(X_embeddings_test_euclidean)

ValueError: ignored

In [80]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [82]:
sents_vectorizer = TfidfVectorizer()
sents_decomp = TruncatedSVD(50)

vectorized = sents_vectorizer.fit_transform(X_texts + X_texts_test)
truncated_sents = sents_decomp.fit_transform(vectorized)

emb_decomp = TruncatedSVD(50)
truncated_embeddings = emb_decomp.fit_transform(np.concatenate([X_embeddings,X_embeddings_test], axis=0))

In [229]:
f1 = sents_decomp.transform(sents_vectorizer.transform(X_texts))
f2 = emb_decomp.transform(X_embeddings)
f3 = X_train_most_words
f4 = X_train_top_words
f5 = X_embeddings_cosine
f6 = X_embeddings_euclidean
f = np.concatenate([f1, f2, f3, f4, f5, f6], axis=1)
print(f.shape)

(11690, 133)


In [230]:
X_train, X_val, y_train, y_val = train_test_split(f, y, test_size=0.33, random_state=42)

In [228]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
knn = KNeighborsClassifier(n_neighbors=3, weights='distance', metric='cosine')
svc = SVC(class_weight='balanced')
ridge = RidgeClassifierCV(class_weight='balanced')

knn.fit(scaler.fit_transform(X_train), y_train)

svc.fit(scaler.transform(X_train), y_train)
ridge.fit(scaler.transform(X_train), y_train)

print("KNN")
y_pred_train = knn.predict(scaler.transform(X_train))
y_pred_val = knn.predict(scaler.transform(X_val))

print(classification_report(y_true=y_train, y_pred=y_pred_train))
print(f1_score(y_true=y_train, y_pred=y_pred_train))

print(classification_report(y_true=y_val, y_pred=y_pred_val))
print(f1_score(y_true=y_val, y_pred=y_pred_val))


print("SVC")
y_pred_train = svc.predict(scaler.transform(X_train))
y_pred_val = svc.predict(scaler.transform(X_val))

print(classification_report(y_true=y_train, y_pred=y_pred_train))
print(f1_score(y_true=y_train, y_pred=y_pred_train))

print(classification_report(y_true=y_val, y_pred=y_pred_val))
print(f1_score(y_true=y_val, y_pred=y_pred_val))


print("Ridge")
y_pred_train = ridge.predict(scaler.transform(X_train))
y_pred_val = ridge.predict(scaler.transform(X_val))

print(classification_report(y_true=y_train, y_pred=y_pred_train))
print(f1_score(y_true=y_train, y_pred=y_pred_train))

print(classification_report(y_true=y_val, y_pred=y_pred_val))
print(f1_score(y_true=y_val, y_pred=y_pred_val))

KNN
              precision    recall  f1-score   support

           0       0.97      0.94      0.96      5587
           1       0.87      0.93      0.90      2245

    accuracy                           0.94      7832
   macro avg       0.92      0.93      0.93      7832
weighted avg       0.94      0.94      0.94      7832

0.8955288048151333
              precision    recall  f1-score   support

           0       0.84      0.81      0.82      2742
           1       0.56      0.62      0.59      1116

    accuracy                           0.75      3858
   macro avg       0.70      0.71      0.71      3858
weighted avg       0.76      0.75      0.75      3858

0.589041095890411
SVC
              precision    recall  f1-score   support

           0       0.92      0.82      0.87      5587
           1       0.65      0.83      0.73      2245

    accuracy                           0.82      7832
   macro avg       0.79      0.83      0.80      7832
weighted avg       0.85      

In [131]:
X_train_extended = np.concatenate([X_train,
                                   knn.predict_proba(scaler.transform(X_train)),
                                   svc.predict(scaler.transform(X_train))[:, None],
                                   ridge.predict(scaler.transform(X_train))[:, None],
                                   ], axis=1)

X_val_extended = np.concatenate([X_val,
                                   knn.predict_proba(scaler.transform(X_val)),
                                   svc.predict(scaler.transform(X_val))[:, None],
                                   ridge.predict(scaler.transform(X_val))[:, None],
                                   ], axis=1)

In [89]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 3.4 MB/s eta 0:00:00


In [231]:
from catboost import CatBoostClassifier, Pool

In [244]:

from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=None, shuffle=False)

models = []
y = np.array(y)
for i, (train_index, test_index) in enumerate(kf.split(f)):
    X_train = f[train_index]
    y_train = y[train_index]
    X_val = f[test_index]
    y_val = y[test_index]

    cb_model = CatBoostClassifier(iterations=500,
                            depth=6,
                            learning_rate=0.03,
                            loss_function='Logloss',
                            auto_class_weights='Balanced',
                            verbose=False)

    cb_model.fit(X_train, y_train, eval_set=(X_val, y_val))
    y_pred_train = cb_model.predict(X_train)
    y_pred_val = cb_model.predict(X_val)

    print(classification_report(y_true=y_train, y_pred=y_pred_train))
    print(f1_score(y_true=y_train, y_pred=y_pred_train))

    print(classification_report(y_true=y_val, y_pred=y_pred_val))
    print(f1_score(y_true=y_val, y_pred=y_pred_val))
    models.append(cb_model)


              precision    recall  f1-score   support

           0       0.96      0.90      0.93      6687
           1       0.79      0.91      0.84      2665

    accuracy                           0.90      9352
   macro avg       0.88      0.90      0.89      9352
weighted avg       0.91      0.90      0.91      9352

0.8439084076210452
              precision    recall  f1-score   support

           0       0.90      0.80      0.85      1642
           1       0.62      0.78      0.69       696

    accuracy                           0.79      2338
   macro avg       0.76      0.79      0.77      2338
weighted avg       0.81      0.79      0.80      2338

0.6930440331844289
              precision    recall  f1-score   support

           0       0.97      0.91      0.94      6759
           1       0.79      0.91      0.85      2593

    accuracy                           0.91      9352
   macro avg       0.88      0.91      0.89      9352
weighted avg       0.92      0.91   

In [233]:
y_pred_train = cb_model.predict(X_train)
y_pred_val = cb_model.predict(X_val)


print(classification_report(y_true=y_train, y_pred=y_pred_train))
print(f1_score(y_true=y_train, y_pred=y_pred_train))

print(classification_report(y_true=y_val, y_pred=y_pred_val))
print(f1_score(y_true=y_val, y_pred=y_pred_val))

              precision    recall  f1-score   support

           0       1.00      0.97      0.98      7503
           1       0.93      0.99      0.96      3018

    accuracy                           0.98     10521
   macro avg       0.96      0.98      0.97     10521
weighted avg       0.98      0.98      0.98     10521

0.9585666293393057
              precision    recall  f1-score   support

           0       0.90      0.89      0.90       826
           1       0.75      0.77      0.76       343

    accuracy                           0.86      1169
   macro avg       0.83      0.83      0.83      1169
weighted avg       0.86      0.86      0.86      1169

0.7593123209169055


In [245]:
f1 = sents_decomp.transform(sents_vectorizer.transform(X_texts_test))
f2 = emb_decomp.transform(X_embeddings_test)
f3 = X_test_most_words
f4 = X_test_top_words
f5 = X_embeddings_test_cosine
f6 = X_embeddings_test_euclidean
f_test = np.concatenate([f1, f2, f3, f4, f5, f6], axis=1)
X_test = f_test
print(X_test.shape)

(16627, 133)


In [268]:
predicts = []

for model in models:
    predicts.append(model.predict(X_test))


predicts = np.vstack(predicts).T

In [269]:
predicts.shape

(16627, 5)

In [277]:
preds = np.sum(predicts, axis=1)

In [279]:
idxs = np.logical_and(preds != 5, preds != 0)

In [281]:
unique, count = np.unique(preds[idxs], return_counts=True)

In [282]:
print(unique, count)

[1 2 3 4] [510 306 324 485]


In [271]:
final_predicts = ((np.sum(predicts, axis=1) / 5) > 0.5).astype(int)

In [274]:
X_test = f_test
df = pd.DataFrame({'pair_id': pairs, 'target': final_predicts})
df.to_csv("cb_ansemble.csv", index=None)